<a href="https://colab.research.google.com/github/palbha/airline-faq-rag/blob/main/faq_data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary libraries if required - This code ran on Google Colab & the libraires where supported by default- please rephrase this



In [2]:
import csv
import json
import os
import openai
import csv
from openai import OpenAI
from google.colab import userdata

#Based on airlines FAQ I identified potential topics which can be shared with our agent to create FAQ's
FAQ_TOPICS = [
    "Airport Services",
    "Animal Transportation",
    "Beyond Business",
    "Booking and managing a reservation",
    "Carbon Offsetting",
    "AirlineX Compliance",
    "Hotels, cars and travel insurance",
    "AirlineX Offers",
    "On-board experience",
    "Operational Updates",
    "Payments",
    "Privilege Club : Qatar Airways' loyalty programme",
    "ArlineX Airways Affiliate Program",
    "ArlineX Airways Packages",
    "ATravel - ArlineX Loyalty Program",
    "ATravel - ArlineX Loyalty Program - Account Cancellation",
    "ATravel - ArlineX Loyalty Program - Account Management",
    "ATravel - ArlineX Loyalty Program - Booking Terms and Conditions",
    "Travel Baggage",
    "Baggage",
    "BAGTAG",
    "Hand baggage",
    "Liquids",
    "Mishandled baggage",
    "Travel voucher",
    "Voucher redemption",
    "TripAdd",
    "eSIM - TripAdd",
    "Lounge - TripAdd",
    "Meet and Greet - TripAdd",
    "Young Travellers",
    "Travelling with children",
    "Unaccompanied minors",
]

In [3]:
openai = OpenAI(
   base_url="https://generativelanguage.googleapis.com/v1beta/",
  api_key=userdata.get('gemini_api'),
)

In [4]:
def get_faq_ques_for_topic(topic):
    messages = [
        {
            "role": "system",
            "content": (
                """You are an assistant that generates FAQ-style questions  for an airline named Airline X, which operates international and domestic flights in Canada.

For each topic, Generate  realistic and informative user-style questions for the FAQ topic. Do no include answers
"""
            )
        },
        {
            "role": "user",
            "content": f"Generate FAQ 5-10 questions  about the topic: '{topic}'."
        },
        {
            "role": "system",
            "content": "Return ONLY a valid JSON array of question objects. Do not include answers"
        }
    ]
    response = openai.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages,
        temperature=0.7,
        max_tokens=700,
    )
    # The model response should be a JSON array of objects like: [{"question": "...", "answer": "..."}, ...]
    content = response.choices[0].message.content.strip()


    return content



In [5]:
def get_faq_ans_for_topic(topic,question):
    messages = [
        {
            "role": "system",
            "content": (
                f"You are an assistant that generates FAQ-style answers for an airline named Airline X, "
        f"which operates international and domestic flights in Canada.\n\n"
        f"The FAQ topic is: '{topic}'.\n"

            )
        },
        {
            "role": "user",
            "content": f"Generate FAQ answers for the question: '{question}'."
        }       ,
        {
            "role": "system",
            "content": "Provide a clear, self-contained, and factual-sounding answer based on Airline X's own policies. "
        "Do NOT reference any website, support, or external links. Make the answer complete, realistic, "
        "and independent of outside context. Return ONLY answers"
        }
    ]
    response = openai.chat.completions.create(
        model="gemini-1.5-flash",
        messages=messages,
        temperature=0.7,
        max_tokens=700,
    )
    # The model response should be a JSON array of objects like: [{"question": "...", "answer": "..."}, ...]
    content = response.choices[0].message.content.strip()


    return content



In [6]:
faq_data = []
import re
for topic in FAQ_TOPICS:
    questions=get_faq_ques_for_topic(topic)
    raw_text = questions.strip()
    cleaned = re.sub(r"^```json|```$", "", raw_text, flags=re.IGNORECASE).strip("`\n ")

    # Now attempt to parse
    question_data = json.loads(cleaned)
    for key in question_data:
      answer=get_faq_ans_for_topic(topic,key['question'])
      faq_data.append({
            "topic": topic,
            "question": key['question'],
            "answer": answer
        })


In [7]:
import pandas as pd
pd.DataFrame(faq_data).to_csv("faq_data.csv",index=False)